Packages réguliers

In [ ]:
import scipy as sc
from scipy.sparse import *
from scipy.sparse.linalg import *
import numpy as np
import pandas as pd
import sys
import os
import matplotlib.pyplot as plt
import gstlearn as gl
import gstlearn.plot as gp

Paramètres d'environnement

In [ ]:
verbose = False

### Création du champ de vecteur

In [ ]:
def fa(x,y,a,b):
    return a*x + b*y

def spirale(db,a=0,b=-1.4,c=1.,d=1.,plot = False):
    x1c = np.array(db.getField("x1")) #getField ou mieux getCoords coords = workingDb.getCoords()
    x2c = np.array(db.getField("x2")) 
    u1=fa(x1c-50,x2c-50,a,b)
    u2=fa(x1c-50,x2c-50,c,d)
    shape = db.getNXs()
    norm = np.sqrt(u1**2+u2**2)
    ind = norm>0
    theta = np.zeros_like(norm)
    theta[norm>0] = np.arccos(u2[ind]/norm[ind])/np.pi*180*np.sign(u1[ind])
    x1c=x1c.reshape(shape)
    x2c=x2c.reshape(shape)
    u1=u1.reshape(shape)
    u2=u2.reshape(shape)
    if plot:
        plt.quiver(x1c,x2c,u1,u2)
        plt.axis("equal")
        plt.show()
    return theta

Draw a vector field

In [ ]:
workingDbc = gl.DbGrid.create([10,10],[10,10])
spirale(workingDbc,plot=True);

Création de la grille de travail

In [ ]:
resultDb = gl.DbGrid.create([200,200],[0.5,0.5]) 

Création de la db des données

In [ ]:
np.random.seed(124)
ndat=10000
coords=np.random.uniform(1,99,size=(ndat,2))
dat = gl.Db()
dat.addFields(coords[:,0],"X")
dat.addFields(coords[:,1],"Y")
dat.setLocator(['X','Y'],gl.ELoc.X)

Création du modèle. Attention le grand axe doit etre fourni en premier: il correspond à la direction pointée par l'angle 'theta'.

In [ ]:
model = gl.Model.createFromDb(resultDb)
cova = gl.CovAniso(gl.ECov.BESSEL_K,model.getContext()) #Alias ECov.MATERN
cova.setRanges([4,45])
model.addCova(cova)
spirale = gl.FunctionalSpirale(0., 1.4, 1., 1., 50., 50.);
nostat = gl.NoStatFunctional(spirale)
err = model.addNoStat(nostat)

Création du meshing (Turbo)

In [ ]:
workingDb = gl.DbGrid.create([101,101],[1,1]) 
mesh = gl.MeshETurbo(workingDb)

Création du Shift Operator

In [ ]:
S = gl.ShiftOpCs(mesh, model, resultDb)

Création de l'opérateur de précision

In [ ]:
Qsimu = gl.PrecisionOp(S, cova, gl.EPowerPT.MINUSHALF, verbose)

Simulation non-conditionnelle

In [ ]:
vect = gl.VectorDouble(np.random.normal(size=Qsimu.getSize()))
result = gl.VectorDouble(np.empty_like(vect))
Qsimu.eval(vect,result)
workingDb.addFields(result,"Simu",gl.ELoc.X)

ax = gp.grid(workingDb,"Simu")

Matrice de précision

In [ ]:
Qkriging = gl.PrecisionOpCs(S, cova, gl.EPowerPT.ONE)
Qtr = gl.csToTriplet(Qkriging.getQ())
Qmat = sc.sparse.csc_matrix((np.array(Qtr.values), (np.array(Qtr.rows), np.array(Qtr.cols))))

## Comparaison produit par Q de 2 façons

In [ ]:
xx=np.random.normal(size=Qkriging.getSize())
vectxx = gl.VectorDouble(xx)

In [ ]:
y=Qmat@xx

In [ ]:
resultxx = gl.VectorDouble(np.empty_like(vectxx))
Qkriging.eval(vectxx,resultxx)

In [ ]:
plt.scatter(resultxx,y,s=1)
plt.show()

### Vérification de l'inverse

In [ ]:
Qtest = gl.PrecisionOp(S, cova, gl.EPowerPT.MINUSONE)
resulttest = gl.VectorDouble(np.empty_like(vectxx))
Qtest.eval(resultxx,resulttest)
plt.scatter(resulttest,xx,s=1)
plt.show()

# Suspect 

Comparaison de $Q^{-1}x$ et $Q^{-1/2}Q^{-1/2}x$

In [ ]:
xx=np.random.normal(size=Qkriging.getSize())
vectxx = gl.VectorDouble(xx)
resultxx2 = gl.VectorDouble(np.empty_like(vectxx))

#Méthode 1
Qsimu.eval(xx,vectxx)
Qsimu.eval(vectxx,resultxx2)

#Méthode 2
Qtest.eval(xx,resulttest)

plt.scatter(resultxx2,resulttest,s=1)
plt.show()

Matrice de projection (on utilise un constructeur specifique)

In [ ]:
B = gl.ProjMatrix(dat,mesh)
Btr = gl.csToTriplet(B.getAproj())
Bmat=sc.sparse.csc_matrix((np.array(Btr.values), (np.array(Btr.rows), np.array(Btr.cols))),
                          shape=(Btr.nrows,Btr.ncols))

Génération des données

In [ ]:
size = dat.getSampleNumber()
u=gl.VectorDouble(np.zeros(size))
B.mesh2point(result,u)
dat.addFields(u,"Z",gl.ELoc.Z)
plt.scatter(coords[:,0],coords[:,1],s=.5,c=dat.getField("Z"),marker="s")
plt.show()
datVal =[i for i in u]

In [ ]:
nug = 0.01
WorkingMat = Qmat+1/nug * Bmat.T @ Bmat
rhs = 1/nug * Bmat.T * datVal
rhsvd = gl.VectorDouble(rhs)

In [ ]:
kriging = sc.sparse.linalg.cg(WorkingMat,rhs)[0] #Ici rebrancher le gradient conjugué

In [ ]:
iatt = workingDb.addFields(kriging,"Kriging")

In [ ]:
ax = gp.grid(workingDb,"Kriging",title="Kriging on Working Grid")

Projection sur la grille de résultats

In [ ]:
Bresult = gl.ProjMatrix(resultDb,mesh)
Bresulttr = gl.csToTriplet(Bresult.getAproj())
Bresultmat=sc.sparse.csc_matrix((np.array(Bresulttr.values), (np.array(Bresulttr.rows), np.array(Bresulttr.cols))),
                          shape=(Bresulttr.nrows,Bresulttr.ncols))

In [ ]:
iatt = resultDb.addFields(Bresultmat@kriging,"Kriging")

In [ ]:
ax = gp.grid(resultDb,"Kriging",title="Kriging on Resulting Grid")

In [ ]:
vc = gl.VectorVectorDouble()
vc.push_back(gl.VectorDouble(rhs))
resultvc = gl.VectorVectorDouble()
resultvc.push_back(gl.VectorDouble(np.zeros_like(rhs)))

# Test de evalDirect

In [ ]:
A=gl.PrecisionOpMultiConditional()
A.push_back(Qkriging,B)
A.setVarianceData(nug)
A.evalDirect(vc,resultvc)

In [ ]:
m=np.min(WorkingMat@rhs)
M=np.max(WorkingMat@rhs)
plt.scatter(WorkingMat@rhs,resultvc[0],s=1)
plt.plot([m,M],[m,M],c="r")
plt.show()
np.max(np.abs(WorkingMat@rhs-resultvc[0]))

# Test de evalInverse

In [ ]:
A.evalInverse(vc,resultvc)
plt.scatter(kriging,resultvc[0],s=1)
plt.show()

In [ ]:
iatt = workingDb.addFields(resultvc[0],"Kriging")
ax = gp.grid(workingDb,"Kriging")

## Calcul du log du déterminant de Q

1) Somme des logs des valeurs propres

In [ ]:
#eigvals=sc.linalg.eigvals(Qmat.todense())

In [ ]:
#np.sum(np.log(np.real(eigvals)))

2) Cholesky (nécessite scikit-sparse basé sur CHOLMOD qui doit être installé)

In [ ]:
from sksparse.cholmod import cholesky
cc=cholesky(Qmat)
cc.logdet()

3) Approximation par la méthode de Mike

In [ ]:
Qlog = gl.PrecisionOp(S, cova, gl.EPowerPT.LOG)

In [ ]:
s=0
nsim = 1000
for i in range(nsim):
    xx=np.array([1.  if i>0 else -1. for i in np.random.normal(size=Qkriging.getSize())])
    xx=np.random.normal(size=Qkriging.getSize())
    Qlog.eval(xx,result)
    s+=np.sum(result*xx)
resc=s/nsim

In [ ]:
v=np.sum(np.log(S.getLambdas()))

In [ ]:
resc+2*v

In [ ]:
Qlog.computeLogDet(1000,1003)

In [ ]:
import numpy as np

from scipy.sparse.linalg import LinearOperator

class prodBlock():
    def __init__(self,Q,Pmat,nugget=0.01,Qinv=None):
        self.Q = Qkriging
        self.P = Pmat
        self.Qinv=Qinv
        self.nugget = nugget
        self.nvertex = self.Q.getSize()
        self.ndata=self.P.getPointNumber()
        n=self.ndata+self.nvertex
        self.shape=[n,n]
       # self.dtype=object
        self.r1 = gl.VectorDouble(np.zeros(self.nvertex))
        self.r2 = gl.VectorDouble(np.zeros(self.ndata))
        self.t1 = gl.VectorDouble(np.zeros(self.nvertex))
        self.t2 = gl.VectorDouble(np.zeros(self.ndata))
    def _matvec(self,v):
        v1 = gl.VectorDouble(v[0:self.nvertex])
        v2 = gl.VectorDouble(v[self.nvertex:(self.nvertex+self.ndata)])
        self.Q.eval(v1,self.r1)
        self.P.point2mesh(v2,self.t1)
        for i in range(self.r1.size()):
            self.r1[i]+=self.t1[i]
        self.P.mesh2point(v1,self.r2)
        for i in range(self.r2.size()):
            self.r2[i]-=self.nugget * v2[i]
        return np.concatenate([np.array(self.r1),np.array(self.r2)])
    def _precond(self,v):
        v1 = gl.VectorDouble(v[0:self.nvertex])
        v2 =v[self.nvertex:(self.nvertex+self.ndata)]
        self.Qinv.eval(v1,self.r1)
        return np.concatenate([np.array(self.r1),v2])
    def _precondDirect(self,v):
        v1 = gl.VectorDouble(v)
        self.Qinv.eval(v1,self.r1)
        return np.array(self.r1)
    def _direct(self,v):
        v1 = gl.VectorDouble(v)
        self.Q.eval(v1,self.r1)
        self.P.mesh2point(v1,self.t2)
        self.P.point2mesh(self.t2,self.t1)
        for i in range(self.r1.size()):
            self.r1[i]+=1/self.nugget*self.t1[i]
        return np.array(self.r1)

In [ ]:
LinOp=prodBlock(Qkriging,B,nug,Qtest)
NewRHS=gl.VectorDouble(np.concatenate((rhs,np.zeros(ndat))))

In [ ]:
LinOp._matvec(NewRHS)

In [ ]:
np.random.seed(0)
NewRHS=gl.VectorDouble(np.concatenate((rhs,np.zeros(ndat))))
LinOp._matvec(NewRHS)
Alin = LinearOperator(LinOp.shape, matvec=LinOp._matvec)
ACG = LinearOperator((Qkriging.getSize(),Qkriging.getSize()), matvec=LinOp._direct)
Precond = LinearOperator(LinOp.shape, matvec=LinOp._precond)
PrecondDirect = LinearOperator((Qkriging.getSize(),Qkriging.getSize()), matvec=LinOp._precondDirect)

In [ ]:
from scipy.sparse.linalg import gmres
u=gmres(Alin, NewRHS,maxiter=10,tol=1e-5,atol=1e-5,M=Precond)
plt.scatter(kriging,u[0][0:Qkriging.getSize()],s=1)
plt.show()

In [ ]:
from scipy.sparse.linalg import lgmres
u=lgmres(Alin, NewRHS,maxiter=15,tol=1e-5,atol=1e-5,M=Precond)
plt.scatter(kriging,u[0][0:Qkriging.getSize()],s=1)
plt.show()

In [ ]:
from scipy.sparse.linalg import minres
u=minres(Alin, NewRHS,maxiter=100,tol=1e-10,M=Precond)
plt.scatter(kriging,u[0][0:Qkriging.getSize()],s=1)
plt.show()

In [ ]:
u=cg(ACG, rhs,maxiter=100,tol=1e-10,atol=1e-10,M=PrecondDirect)
plt.scatter(u[0],kriging,s=1)
plt.show()

In [ ]:
u=cg(Alin, NewRHS,maxiter=10,tol=1e-10,atol=1e-10,M=Precond)
plt.scatter(kriging,u[0][0:Qkriging.getSize()],s=1)
plt.show()